In [33]:
import pandas as pd
import pyspark as spark
from pyspark.sql import SparkSession

In [34]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [35]:
df = spark.read\
    .option("header","true")\
    .parquet("data/raw/green/2020/01")

In [36]:
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [37]:
# Read as pandas 
df_pandas = pd.read_parquet("data/raw/green/2020/01", engine="pyarrow")
df_pandas.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-12-18 15:52:30,2019-12-18 15:54:39,N,1.0,264,264,5.0,0.00,3.5,0.50,0.5,0.01,0.0,None,0.3,4.81,1.0,1.0,0.00
1,2,2020-01-01 00:45:58,2020-01-01 00:56:39,N,5.0,66,65,2.0,1.28,20.0,0.00,0.0,4.06,0.0,None,0.3,24.36,1.0,2.0,0.00
2,2,2020-01-01 00:41:38,2020-01-01 00:52:49,N,1.0,181,228,1.0,2.47,10.5,0.50,0.5,3.54,0.0,None,0.3,15.34,1.0,1.0,0.00
3,1,2020-01-01 00:52:46,2020-01-01 01:14:21,N,1.0,129,263,2.0,6.30,21.0,3.25,0.5,0.00,0.0,None,0.3,25.05,2.0,1.0,2.75
4,1,2020-01-01 00:19:57,2020-01-01 00:30:56,N,1.0,210,150,1.0,2.30,10.0,0.50,0.5,0.00,0.0,None,0.3,11.30,1.0,1.0,0.00


In [38]:
# Check the shape of the dataframe
df_pandas.shape

(447770, 20)

In [39]:
# Some of the types cannot be determined after inferring
df_pandas.isnull().sum()

VendorID                      0
lpep_pickup_datetime          0
lpep_dropoff_datetime         0
store_and_fwd_flag       116051
RatecodeID               116051
PULocationID                  0
DOLocationID                  0
passenger_count          116051
trip_distance                 0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
ehail_fee                447770
improvement_surcharge         0
total_amount                  0
payment_type             116051
trip_type                116052
congestion_surcharge     116051
dtype: int64

In [40]:
# Drop some columns
df_pandas["store_and_fwd_flag"].value_counts()

store_and_fwd_flag
N    330676
Y      1043
Name: count, dtype: int64

In [41]:
# Check the data types of the constituent columns
df_pandas.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
dtype: object

In [42]:
# Ideally, I would like to do a ton of analysis to study AND infer the relationship between the variables before imputing them. But for my purposes here I am just doing random stuff

df_pandas["store_and_fwd_flag"] = df_pandas["store_and_fwd_flag"].fillna("Missing")


**Observations**
- To transform("store_and_fwd_flag")

In [43]:
df_pandas['store_and_fwd_flag'].value_counts()

store_and_fwd_flag
N          330676
Missing    116051
Y            1043
Name: count, dtype: int64

In [44]:
store_fwd_flag_dict = {"N": 'N', "Missing": 'M', "Y": 'Y'}
df_pandas['store_and_fwd_flag'] = df_pandas['store_and_fwd_flag'].map(store_fwd_flag_dict)
df_pandas

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-12-18 15:52:30,2019-12-18 15:54:39,N,1.0,264,264,5.0,0.00,3.50,0.50,0.5,0.01,0.0,None,0.3,4.81,1.0,1.0,0.00
1,2,2020-01-01 00:45:58,2020-01-01 00:56:39,N,5.0,66,65,2.0,1.28,20.00,0.00,0.0,4.06,0.0,None,0.3,24.36,1.0,2.0,0.00
2,2,2020-01-01 00:41:38,2020-01-01 00:52:49,N,1.0,181,228,1.0,2.47,10.50,0.50,0.5,3.54,0.0,None,0.3,15.34,1.0,1.0,0.00
3,1,2020-01-01 00:52:46,2020-01-01 01:14:21,N,1.0,129,263,2.0,6.30,21.00,3.25,0.5,0.00,0.0,None,0.3,25.05,2.0,1.0,2.75
4,1,2020-01-01 00:19:57,2020-01-01 00:30:56,N,1.0,210,150,1.0,2.30,10.00,0.50,0.5,0.00,0.0,None,0.3,11.30,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447765,2,2020-01-31 23:29:00,2020-01-31 23:47:00,M,NaN,167,32,NaN,4.58,23.21,2.75,0.0,0.00,0.0,None,0.3,26.26,NaN,NaN,NaN
447766,2,2020-01-31 23:57:00,2020-02-01 00:23:00,M,NaN,81,69,NaN,6.55,27.27,2.75,0.0,0.00,0.0,None,0.3,30.32,NaN,NaN,NaN
447767,2,2020-01-31 23:57:00,2020-02-01 00:10:00,M,NaN,244,241,NaN,3.34,25.95,2.75,0.0,0.00,0.0,None,0.3,29.00,NaN,NaN,NaN
447768,2,2020-01-31 23:27:00,2020-02-01 00:04:00,M,NaN,68,17,NaN,8.92,30.39,2.75,0.0,0.00,0.0,None,0.3,33.44,NaN,NaN,NaN


In [45]:
# Next null variable which is 'rateCodeId'
df_pandas["RatecodeID"] = df_pandas["RatecodeID"].fillna(0)

In [46]:
# Check for the passenger count as well 
df_pandas["passenger_count"] = df_pandas["passenger_count"].fillna(0)

In [47]:
# Drop ehail_fee
df_pandas = df_pandas.drop("ehail_fee", axis=1)

In [48]:
# Check for 'payment_type'
df_pandas['payment_type'] = df_pandas['payment_type'].fillna(0)

In [49]:
# Check for trip_type
df_pandas['trip_type'] = df_pandas['trip_type'].fillna(0)

In [50]:
# Check for congestion surcharge
df_pandas['congestion_surcharge'] = df_pandas['congestion_surcharge'].fillna(0)

In [51]:
# Now check for nulls again
df_pandas.isnull().sum()

VendorID                 0
lpep_pickup_datetime     0
lpep_dropoff_datetime    0
store_and_fwd_flag       0
RatecodeID               0
PULocationID             0
DOLocationID             0
passenger_count          0
trip_distance            0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
payment_type             0
trip_type                0
congestion_surcharge     0
dtype: int64

In [52]:
# Now check for data types
df_pandas.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
dtype: object

In [53]:
df_pandas['tolls_amount'].value_counts()

tolls_amount
0.00     423498
6.12      19912
2.29       1526
2.80       1257
12.24       616
          ...  
31.99         1
23.00         1
29.62         1
30.99         1
14.99         1
Name: count, Length: 94, dtype: int64

- Honestly, I will it doesn't hurt much to round off the values to their nearest integers

In [54]:
df_pandas['improvement_surcharge']

0         0.3
1         0.3
2         0.3
3         0.3
4         0.3
         ... 
447765    0.3
447766    0.3
447767    0.3
447768    0.3
447769    0.3
Name: improvement_surcharge, Length: 447770, dtype: float64

In [55]:
df_pandas['passenger_count'] = df_pandas['passenger_count'].astype('uint8')

In [56]:
df_pandas['total_amount'].value_counts()

total_amount
8.30      12233
7.80      10143
7.30       9940
6.80       9822
8.80       9024
          ...  
103.49        1
61.36         1
75.54         1
57.72         1
56.74         1
Name: count, Length: 7072, dtype: int64

In [57]:
df_pandas['payment_type']

0         1.0
1         1.0
2         1.0
3         2.0
4         1.0
         ... 
447765    0.0
447766    0.0
447767    0.0
447768    0.0
447769    0.0
Name: payment_type, Length: 447770, dtype: float64

In [58]:
df_pandas['trip_type']

0         1.0
1         2.0
2         1.0
3         1.0
4         1.0
         ... 
447765    0.0
447766    0.0
447767    0.0
447768    0.0
447769    0.0
Name: trip_type, Length: 447770, dtype: float64

In [59]:
df_pandas['congestion_surcharge']

0         0.00
1         0.00
2         0.00
3         2.75
4         0.00
          ... 
447765    0.00
447766    0.00
447767    0.00
447768    0.00
447769    0.00
Name: congestion_surcharge, Length: 447770, dtype: float64

In [60]:
# Create the dataframe
spark.createDataFrame(df_pandas).schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime', TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [62]:
from pyspark.sql import types

In [63]:
green_schema = types.StructType([
    types.StructField('VendorID', types.LongType(), True), 
    types.StructField('lpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('lpep_dropoff_datetime', types.TimestampType(), True), 
    types.StructField('store_and_fwd_flag', types.StringType(), True), 
    types.StructField('RatecodeID', types.IntegerType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID',types.IntegerType(), True), 
    types.StructField('passenger_count', types.LongType(), True), 
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('fare_amount', types.DoubleType(), True), 
    types.StructField('extra', types.DoubleType(), True), 
    types.StructField('mta_tax', types.DoubleType(), True), 
    types.StructField('tip_amount', types.DoubleType(), True), 
    types.StructField('tolls_amount', types.DoubleType(), True),
    types.StructField('improvement_surcharge', types.DoubleType(), True), 
    types.StructField('total_amount', types.DoubleType(), True), 
    types.StructField('payment_type', types.IntegerType(), True), 
    types.StructField('trip_type', types.IntegerType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True)
  ])

In [55]:
df_green_spark = spark.read\
    .option("header","true")\
    .schema(green_schema)\
    .parquet("data/raw/green/2020/01")

In [56]:
df_green_spark.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [57]:
df_green_spark.show()

25/01/25 23:09:13 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 46)
org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/data/raw/green/2020/01/green_tripdata_2020_01.parquet. Column: [RatecodeID], Expected: int, Found: DOUBLE.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodeg

Py4JJavaError: An error occurred while calling o269.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 46) (192.168.0.66 executor driver): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/data/raw/green/2020/01/green_tripdata_2020_01.parquet. Column: [RatecodeID], Expected: int, Found: DOUBLE.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expression

In [71]:
df_yellow_pandas = pd.read_parquet('data/raw/yellow/2020/01/', engine='pyarrow')

In [65]:
yellow_schema = green_schema

In [51]:
# df_yellow_spark = spark.read\
#     .option("header","true")\
#     .schema(yellow_schema)\
#     .parquet("data/raw/yellow/2020/01")

In [66]:
# df_yellow_spark.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime', TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [68]:
df_yellow_pandas.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [72]:
df_yellow_pandas.isnull().sum()

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count            65441
trip_distance                  0
RatecodeID                 65441
store_and_fwd_flag         65441
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge       65441
airport_fee              6405008
dtype: int64

In [67]:
df_yellow_pandas.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                      object
dtype: object

In [69]:
df_yellow_pandas['airport_fee']

0          None
1          None
2          None
3          None
4          None
           ... 
6405003    None
6405004    None
6405005    None
6405006    None
6405007    None
Name: airport_fee, Length: 6405008, dtype: object

In [68]:
df_yellow_pandas['airport_fee']

0          None
1          None
2          None
3          None
4          None
           ... 
6405003    None
6405004    None
6405005    None
6405006    None
6405007    None
Name: airport_fee, Length: 6405008, dtype: object

In [73]:
df_yellow_pandas["store_and_fwd_flag"] = df_yellow_pandas["store_and_fwd_flag"].fillna("Missing")
stoyellow_re_fwd_flag_dict = {"N": 'N', "Missing": 'M', "Y": 'Y'}
df_yellow_pandas['store_and_fwd_flag'] = df_yellow_pandas['store_and_fwd_flag'].map(store_fwd_flag_dict)
# Nyellow_ext null variable which is 'rateCodeId'
df_yellow_pandas["RatecodeID"] = df_yellow_pandas["RatecodeID"].fillna(0)
# Cyellow_heck for the passenger count as well 
df_yellow_pandas["passenger_count"] = df_yellow_pandas["passenger_count"].fillna(0)
df_yellow_pandas['passenger_count'] = df_yellow_pandas['passenger_count'].astype('uint8')
# Dyellow_rop ehail_fee
# df_yellow_pandas = df_yellow_pandas.drop("ehail_fee", axis=1)
# Cyellow_heck for 'payment_type'
df_yellow_pandas['payment_type'] = df_yellow_pandas['payment_type'].fillna(0)
# Cyellow_heck for trip_typeyellow_
# df_yellow_pandas[''] = df_yellow_pandas['trip_type'].fillna(0)
# Cyellow_heck for congestion surchargeyellow_
df_yellow_pandas['congestion_surcharge'] = df_yellow_pandas['congestion_surcharge'].fillna(0)

In [74]:
df_yellow_pandas = df_yellow_pandas.drop("airport_fee", axis=1)

In [75]:
df_yellow_pandas.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
dtype: int64

In [76]:
spark.createDataFrame(df_yellow_pandas).schema

KeyboardInterrupt: 

In [52]:
df_yellow_spark.repartition(4)\
    .write.parquet('data/pq/yellow/01/')

25/01/25 22:58:23 ERROR Executor: Exception in task 5.0 in stage 4.0 (TID 29)
org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/data/raw/yellow/2020/01/yellow_tripdata_2020_01.parquet. Column: [RatecodeID], Expected: int, Found: DOUBLE.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCod

Py4JJavaError: An error occurred while calling o248.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 4.0 failed 1 times, most recent failure: Lost task 5.0 in stage 4.0 (TID 29) (192.168.0.66 executor driver): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/data/raw/yellow/2020/01/yellow_tripdata_2020_01.parquet. Column: [RatecodeID], Expected: int, Found: DOUBLE.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions

In [49]:
year = 2020
for month in range(1, 13):
    print(f'processing data for {year}/{month}')
    
    input_path = f'data/raw/yellow/{year}/{month:02d}/yellow_tripdata_{year}_{month:02d}.parquet'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'
    
        
    df_yellow_spark = spark.read\
    .option("header","true")\
    .schema(yellow_schema)\
    .parquet(input_path)
    
    df_yellow_spark.repartition(4)\
        .write.parquet(output_path)

processing data for 2020/1


25/01/25 22:50:57 ERROR Executor: Exception in task 5.0 in stage 3.0 (TID 18)
org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/data/raw/yellow/2020/01/yellow_tripdata_2020_01.parquet. Column: [RatecodeID], Expected: int, Found: DOUBLE.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCod

Py4JJavaError: An error occurred while calling o230.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 3.0 failed 1 times, most recent failure: Lost task 5.0 in stage 3.0 (TID 18) (192.168.0.66 executor driver): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///Applications/saggydev/projects_learning/data_engineering_course/05_batch_processing/data/raw/yellow/2020/01/yellow_tripdata_2020_01.parquet. Column: [RatecodeID], Expected: int, Found: DOUBLE.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:855)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:290)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions

In [ ]:
# Now I will be rewriting these dataframes to parquet format so that the schema is ultimately saved 
df_yellow_spark.repartition(4).write.parquet('data/pq/yellow/2021/01/')